In [6]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

In [7]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
!ls /home/aistudio/work

In [8]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries

mkdir: cannot create directory ‘/home/aistudio/external-libraries’: File exists
Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 102kB 12.4MB/s ta 0:00:01


In [9]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

In [10]:
relation2iddic = {}
entity2iddic = {}
relation2vecdic = {}
entity2vecdic = {}

with open('TransE/data/relation2id.txt', 'r') as fr:
    lines = fr.readlines()
    for line in lines:
        list = line.split("\t")
        if len(list) != 1:
            relation2iddic[list[0]] = int(list[1].strip())
            relation2iddic[list[1].strip()] = list[0]
print(relation2iddic['P36'])

with open('TransE/data/entity2id.txt', 'r') as fr:
    lines = fr.readlines()
    for line in lines:
        list = line.split("\t")
        if len(list) != 1:
            entity2iddic[list[0]] = int(list[1].strip())
            entity2iddic[list[1].strip()] = list[0]
print(entity2iddic['Q30'])


20
118


请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 

In [74]:
import numpy as np

relationvec = np.loadtxt('TransE/relation2vec.txt')
print(relationvec.shape)
entityvec = np.loadtxt('TransE/entity2vec.txt')
print(entityvec.shape)

(378, 50)
(50000, 50)


In [12]:
def cal_sim(v1, v2, ord = 1):
    return np.linalg.norm(v1 - v2, ord=1)

In [13]:
def get_similar_multi(v1: list, v2: list, ord=1):
    res = []
    v1 = (v1 - np.mean(v1, axis=0)) / np.std(v1, axis=0)
    for v in v2:
        v = (v - np.mean(v, axis=0)) / np.std(v, axis=0)
        res.append(cal_sim(v1, v, ord=1))
    return res

In [75]:
P36vec = relationvec[relation2iddic['P36']]
Q30vec = entityvec[entity2iddic['Q30']]
Q49vec = entityvec[entity2iddic['Q49']]
P36vec = (P36vec - np.mean(P36vec)) / np.std(P36vec)
Q30vec = (Q30vec - np.mean(Q30vec)) / np.std(Q30vec)
Q49vec = (Q49vec - np.mean(Q49vec)) / np.std(Q49vec)
print(P36vec)
print(Q30vec)
print(Q49vec)

[ 1.4317649   0.15170422 -0.26251881 -0.30571097 -1.25556568  1.45044169
 -0.38311542  1.67850702  2.3370622  -0.54868022 -0.5460969  -0.69543763
  0.19987306 -0.31033149 -0.2685877  -0.78048993 -0.88355894 -0.78949354
 -1.20745383 -1.79875498 -0.09622983 -0.36452648  0.48126911 -0.57650071
  0.97754561 -0.09094686  0.39709295 -0.30634493 -0.11954377  0.56629054
  0.58427876 -0.53579214  4.18289718 -0.06981734  0.041429   -0.66050353
 -0.26386033  1.32515096  0.27030636 -0.49735347 -1.41350638  0.75687169
 -0.65513508 -0.18119665 -0.49339302 -0.4586085   0.08667736 -0.00769196
  0.72564636 -0.81806195]
[ 6.22073534e-02 -1.12702153e-01 -1.22723958e+00 -2.60108631e+00
 -1.92146125e-01 -6.24119525e-01 -1.06843448e-02  7.02000005e-02
  5.71284118e-01 -1.58476751e-01  6.05325891e-01  1.00169992e+00
  1.14684732e+00 -2.14152555e-02  4.97264926e-02  4.82835222e-01
 -4.60829440e-02  1.49195274e+00  9.57452480e-01  1.12645348e+00
 -1.67374171e+00  1.66114868e+00 -3.54857794e-01  3.20045764e+00


In [73]:
import heapq
array = get_similar_multi(Q30vec+P36vec, entityvec, ord = 1)
max_indexs = heapq.nsmallest(5, range(len(array)), array.__getitem__)
print(max_indexs)

print([entity2iddic[str(indexs)] for indexs in max_indexs])

[118, 3729, 8505, 391, 7482]
['Q30', 'Q65', 'Q1183', 'Q64', 'Q16557']


In [71]:
array = get_similar_multi(Q30vec-Q49vec, relationvec, ord = 1)
max_indexs = heapq.nsmallest(5, range(len(array)), array.__getitem__)
print(max_indexs)

print([relation2iddic[str(indexs)] for indexs in max_indexs])

[179, 18, 312, 366, 91]
['P1532', 'P17', 'P1336', 'P1408', 'P495']
